In [45]:
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
from sklearn.model_selection import train_test_split
import pandas as pd
import torch

In [2]:
df = pd.read_csv('preprocessed.tsv', sep='\t')
df.head()

,Unnamed: 0.1,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,0,"if Alkar floods her with her mental waste, it ...","If Alkar is flooding her with psychic waste, t...",0.785171,0.010309,0.981983,0.014195
1,1,1,you're becoming disgusting.,Now you're getting nasty.,0.749687,0.071429,0.999039,0.065473
2,2,2,"well, we can spare your life.","Well, we could spare your life, for one.",0.919051,0.268293,0.985068,0.213313
3,3,3,"monkey, you have to wake up.","Ah! Monkey, you've got to snap out of it.",0.664333,0.309524,0.994215,0.053362
4,4,4,I have orders to kill her.,I've got orders to put her down.,0.726639,0.181818,0.999348,0.009402


In [ ]:
df.sample(frac=0.8, random_state=13)

In [17]:
df = df.rename(columns={"reference": "input_text", "translation": "target_text"})
train, test = train_test_split(df, test_size=0.2)
train = pd.DataFrame(train, columns = ['input_text','target_text'])
test = pd.DataFrame(test, columns = ['input_text','target_text'])
train.head()

,input_text,target_text
85,You're arrogant. You have the sickness that el...,you are overwhelmed by the disease that elites...
65,why? Because some stupid college counselor tol...,"Why, because some jaded college counselor said..."
90,"the bitch, she thought bitterly, probably turn...","The sloven, she thought bitterly, has probably..."
49,"I dabble in my chest and back a week, a month ...","Then I wax chest and back weekly, sack and cra..."
86,I should have taken that crummy cashier's job.,I should have taken that lousy job as a cashier.


In [18]:
test.head()

,input_text,target_text
93,I can't believe Okwe never fucked you.,I can't believe Okwe never drove you.
57,I don't need a mountain for chumps.,I don't need a bunny hill.
91,"Damn, I'm glad you're back in town. Yeah, me too.",I'm damn glad you're back in town.
39,he's ashamed of you because you have a blob in...,"He's ashamed of you, because your copybook is ..."
88,We're gonna send you a rape whistle. Huh?,"we're gonna send you an anti-rape whistle, okay?"


In [43]:
model_args = Seq2SeqArgs()
model_args.eval_batch_size = 64
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_seq_length = 128
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.train_batch_size = 8
model_args.use_multiprocessing = False

model_args.do_sample = True
model_args.num_beams = 1
model_args.num_return_sequences = 1
model_args.max_length = 128
model_args.top_k = 50
model_args.top_p = 0.95
model_args.output_dir = "./models"


In [47]:
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
    use_cuda=False,
    use_mps_device = True
)
model.train_model(train, eval_data=test)

  0%|          | 0/80 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/10 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/20 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/strangetany/opt/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/Users/strangetany/opt/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:437: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `2.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


(10,
 {'global_step': [10],
  'eval_loss': [8.733986854553223],
  'train_loss': [9.102274894714355]})

In [49]:
preds = model.predict(test["input_text"][:5].tolist())

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/strangetany/opt/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/Users/strangetany/opt/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:437: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `2.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


In [50]:
preds

['I want to believe Okwe never fucked you.',
 "Why didn't the mountain provide a pass for sledging.",
 "Loyalty to anybody: I'm Glad you're back in town, Mr. Clinton. Yeah, me too.",
 "He's ashamed of you because you have a blob in your life.",
 'If I had a rape whistle, would you call me a feminist, what should I do?']

In [33]:
test["input_text"][:5]

93               I can't believe Okwe never fucked you.
57                  I don't need a mountain for chumps.
91    Damn, I'm glad you're back in town. Yeah, me too.
39    he's ashamed of you because you have a blob in...
88            We're gonna send you a rape whistle. Huh?
Name: input_text, dtype: object